In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
#PATH = '/content/drive/My Drive/PPM_Stability/'
#PATH = "C:/Users/velmurug/Documents/Stability Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/Mythreyi/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
PATH = "C:/Users/mythr/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
#PATH = "C:/Users/n9455647/Documents/GitHub/Stability-Experiments/benchmark_interpretability/PPM_Stability/"
sys.path.append(PATH)

In [2]:
!pip show tensorflow

Name: tensorflow
Version: 2.2.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\programdata\anaconda3\lib\site-packages
Requires: tensorflow-estimator, scipy, tensorboard, opt-einsum, termcolor, keras-preprocessing, six, protobuf, gast, grpcio, absl-py, astunparse, google-pasta, wheel, numpy, wrapt, h5py
Required-by: alibi


In [3]:
from DatasetManager import DatasetManager

import pandas as pd
import numpy as np

import os
import sys
from sys import argv
import pickle
import random
import joblib

from sklearn.metrics import accuracy_score
import tensorflow as tf
tf.keras.compat.disable_v2_backend()
from tensorflow.keras.models import load_model


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import statistics
import math

import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

ImportError: cannot import name 'export_saved_model' from 'tensorflow.python.keras.saving.saved_model' (C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\saving\saved_model\__init__.py)

In [ ]:
datasets = ["production", "sepsis_cases", "bpic2012"]
methods = ["xgboost_single_agg", "xgboost_prefix_agg", "xgboost_prefix_index", "lstm_single_3d"]
dataset_names = {"production":"production", "sepsis_cases":"sepsis_cases_1", "bpic2012":"bpic2012_accepted"}


for data in datasets:
    print(data)
            
    dataset_manager = DatasetManager(dataset_names[data])

    
    for method in methods:
        print(method)
        folder_loc = os.path.join(PATH, "%s/%s/" %(data, method))
        
        if "single" in method:
            num_buckets = 1
        else:
            num_buckets = len([name for name in os.listdir(os.path.join(folder_loc,'models/'))])
            
        for bucket in range(num_buckets):
            if "lstm" in method:
                bucketID = "all"
            else:
                bucketID = bucket+1
            print ('Bucket', bucketID)
            
            X_train_path = os.path.join(folder_loc, "train_data/bucket_%s_prefixes.pickle" % (bucketID))
            with open (X_train_path, 'rb') as f:
                dt_train_bucket = pickle.load(f)

            X_test_path = os.path.join(folder_loc, "test_data/bucket_%s_prefixes.pickle" % (bucketID))
            with open (X_test_path, 'rb') as f:
                dt_test_bucket = pickle.load(f)
            

            if "lstm" not in method: 
                y_train = dataset_manager.get_label_numeric(dt_train_bucket)
                y_test = dataset_manager.get_label_numeric(dt_test_bucket)
            
                train_pos_cases = len([i for i in y_train if i == 1])
                train_neg_cases = len([i for i in y_train if i == 0])
                train_pos_ratio = train_pos_cases/(train_pos_cases + train_neg_cases)
                print("\tPositives in training set:", train_pos_ratio)

                test_pos_cases = len([i for i in y_test if i == 1])
                test_neg_cases = len([i for i in y_test if i == 0])
                test_pos_ratio = test_pos_cases/(test_pos_cases + test_neg_cases)
                print("\tPositives in testing set:", test_pos_ratio)
                
                feat_comb_path = os.path.join(folder_loc, "bucketers_and_encoders/feature_combiner_bucket_%s.joblib" % (bucketID))
                with open (feat_comb_path, 'rb') as f:
                    feature_combiner = joblib.load(f)
                
                train_x = feature_combiner.transform(dt_train_bucket)
                test_x = feature_combiner.transform(dt_test_bucket)
                vector_length = test_x[0].shape
                
                print("Training instances:", len(y_train))
                print("Testing instances:", len(y_test))
                print("Feature Vector Length:", vector_length)
                
            else:
                y_train_path = os.path.join(folder_loc, "train_data/bucket_%s_labels.pickle" % (bucketID))
                with open (y_train_path, 'rb') as f:
                    train_y = pickle.load(f)

                y_test_path = os.path.join(folder_loc, "test_data/bucket_%s_labels.pickle" % (bucketID))
                with open (y_test_path, 'rb') as f:
                    test_y = pickle.load(f)
                
                y_train = [int(np.where(i==1)[0]) for i in train_y]
                y_test = test_y
               
                train_pos_cases = len([i for i in y_train if i == 1])
                train_neg_cases = len([i for i in y_train if i == 0])
                train_pos_ratio = train_pos_cases/(train_pos_cases + train_neg_cases)
                print("\tPositives in training set:", train_pos_ratio)

                test_pos_cases = len([i for i in y_test if i == 1])
                test_neg_cases = len([i for i in y_test if i == 0])
                test_pos_ratio = test_pos_cases/(test_pos_cases + test_neg_cases)
                print("\tPositives in testing set:", test_pos_ratio)
                
                vector_length = dt_test_bucket[0].shape
                
                print("Training instances:", len(y_train))
                print("Testing instances:", len(y_test))
                print("Feature Vector Length:", vector_length)

In [ ]:
datasets = ["production", "sepsis_cases", "bpic2012"]
methods = ["xgboost_single_agg", "xgboost_prefix_agg", "xgboost_prefix_index", "lstm_single_3d"]
dataset_names = {"production":"production", "sepsis_cases":"sepsis_cases_1", "bpic2012":"bpic2012_accepted"}


for data in datasets:
    print(data)
            
    dataset_manager = DatasetManager(dataset_names[data])

    
    for method in methods:
        print(method)
        folder_loc = os.path.join(PATH, "%s/%s/" %(data, method))
        
        if "single" in method:
            num_buckets = 1
        else:
            num_buckets = len([name for name in os.listdir(os.path.join(folder_loc,'models/'))])
            
        train_lengths = []
        train_predictions = []
        train_y_all = []
        
        test_lengths = []
        test_predictions = []
        test_y_all = []
            
        for bucket in range(num_buckets):
            if "lstm" in method:
                bucketID = "all"
            else:
                bucketID = bucket+1
            print ('Bucket', bucketID)
            
            X_train_path = os.path.join(folder_loc, "train_data/bucket_%s_prefixes.pickle" % (bucketID))
            with open (X_train_path, 'rb') as f:
                dt_train_bucket = pickle.load(f)

            X_test_path = os.path.join(folder_loc, "test_data/bucket_%s_prefixes.pickle" % (bucketID))
            with open (X_test_path, 'rb') as f:
                dt_test_bucket = pickle.load(f)
            

            if "lstm" not in method: 
                y_train = dataset_manager.get_label_numeric(dt_train_bucket)
                y_test = dataset_manager.get_label_numeric(dt_test_bucket)
            
                train_pos_cases = len([i for i in y_train if i == 1])
                train_neg_cases = len([i for i in y_train if i == 0])
                train_pos_ratio = train_pos_cases/(train_pos_cases + train_neg_cases)
                print("\tPositives in training set:", train_pos_ratio)

                test_pos_cases = len([i for i in y_test if i == 1])
                test_neg_cases = len([i for i in y_test if i == 0])
                test_pos_ratio = test_pos_cases/(test_pos_cases + test_neg_cases)
                print("\tPositives in testing set:", test_pos_ratio)
                
                pipeline_path = os.path.join(folder_loc, "pipelines/pipeline_bucket_%s.joblib" % (bucketID))
                with open (pipeline_path, 'rb') as f:
                    pipeline = joblib.load(f)
                
                prefix_lengths = dataset_manager.get_prefix_lengths(dt_train_bucket)             
                train_lengths.extend(prefix_lengths.values)
                
                prefix_lengths = dataset_manager.get_prefix_lengths(dt_test_bucket)             
                test_lengths.extend(prefix_lengths.values)
                
                train_preds = pipeline.predict(dt_train_bucket)
                train_predictions.extend(train_preds)
                
                test_preds = pipeline.predict(dt_test_bucket)
                test_predictions.extend(test_preds)
                
                train_y_all.extend(y_train)
                test_y_all.extend(y_test)
                
            else:
                y_train_path = os.path.join(folder_loc, "train_data/bucket_%s_labels.pickle" % (bucketID))
                with open (y_train_path, 'rb') as f:
                    train_y = pickle.load(f)

                y_test_path = os.path.join(folder_loc, "test_data/bucket_%s_labels.pickle" % (bucketID))
                with open (y_test_path, 'rb') as f:
                    test_y = pickle.load(f)
                
                y_train = [int(np.where(i==1)[0]) for i in train_y]
                y_test = test_y
                
                cls_path = os.path.join(folder_loc, "cls/pred_cls.h5")
                cls = load_model(cls_path)
                
                preds = cls.predict(dt_train_bucket)
                train_preds = [np.argmax(i, axis = -1) for i in preds]
                
                preds = cls.predict(dt_test_bucket)
                test_preds = [np.argmax(i, axis = -1) for i in preds]
            
        lens = list(set(train_lengths))
        train_accs = []
        test_accs = []
        for i in lens:
            idxs = [n for n in range(len(train_lengths)) if train_lengths[n] == i]
            preds = [train_predictions[n] for n in idxs]
            actual = [train_y_all[n] for n in idxs]
            accuracy = accuracy_score(actual, preds)
            train_accs.append(accuracy)
            
            idxs = [n for n in range(len(test_lengths)) if test_lengths[n] == i]
            preds = [test_predictions[n] for n in idxs]
            actual = [test_y_all[n] for n in idxs]
            accuracy = accuracy_score(actual, preds)
            test_accs.append(accuracy)


        plt.plot(lens, train_accs, label = 'train')
        plt.plot(lens, test_accs, label = 'test')
        plt.legend(loc = "upper left")
        plt.title(data+method)
        plt.show()
            
        #break
                